In [ ]:
USE {
  dependencies {
    implementation("io.github.p-org.solvers:z3:4.8.14-v5")
  }
}

In [ ]:
import java.io.File

data class Hailstone(
    val sx: Long,
    val sy: Long,
    val sz: Long,
    val vx: Long,
    val vy: Long,
    val vz: Long,
    val a: Long,
    val b: Long,
    val c: Long
) {
    constructor(
        sx: Long,
        sy: Long,
        sz: Long,
        vx: Long,
        vy: Long,
        vz: Long,
    ) : this(
        sx, sy, sz, vx, vy, vz, vy, -vx, (vy * sx) - (vx * sy)
    )
}

val hailstones = File("input.txt").readLines().map { line ->
    val values = line.replace("@", ",").split(",").map { it.trim(' ').toLong() }
    Hailstone(values[0], values[1], values[2], values[3], values[4], values[5])
}


hailstones.foldIndexed(0) { i, acc, hs1 ->
    acc + hailstones.subList(0, i).fold(0) { innerAcc, hs2 ->
        val a1 = hs1.a
        val b1 = hs1.b
        val c1 = hs1.c
        val a2 = hs2.a
        val b2 = hs2.b
        val c2 = hs2.c

        if (a1 * b2 == b1 * a2) return@fold innerAcc

        val x = (c1.toDouble() * b2 - c2.toDouble() * b1) / (a1.toDouble() * b2 - a2.toDouble() * b1)
        val y = (c2.toDouble() * a1 - c1.toDouble() * a2) / (a1.toDouble() * b2 - a2.toDouble() * b1)

        val xRange = 200000000000000.0..400000000000000.0
        val yRange = 200000000000000.0..400000000000000.0


        if (x in xRange && y in yRange && listOf(hs1, hs2)
                .all { (x - it.sx) * it.vx >= 0 && (y - it.sy) * it.vy >= 0 }
        ) {
            return@fold innerAcc + 1
        }
        innerAcc
    }
}

In [ ]:
import com.microsoft.z3.*

operator fun <T> List<T>.component6(): T = get(5)

data class Point3D(val x: Double, val y: Double, val z: Double)
data class Velocity3D(val x: Double, val y: Double, val z: Double)
data class Hail(val p: Point3D, val v: Velocity3D) {
    fun slope() = v.y / v.x
}

val hail = File("input.txt").readLines().map {
    it.split(" @ ").let {
        Hail(it[0].split(", ").map { it.trim().toDouble() }.let { (x, y, z) -> Point3D(x, y, z) },
            it[1].split(", ").map { it.trim().toDouble() }.let { (x, y, z) -> Velocity3D(x, y, z) })
    }
}
val ctx = Context() // if using proof = true, Real errors
val solver = ctx.mkSolver()

val (x, y, z, vx, vy, vz) = listOf("x", "y", "z", "vx", "vy", "vz").map { ctx.mkRealConst(it) }
(0..2).forEach { idx ->
    val h = hail[idx]
    val t = ctx.mkRealConst("t$idx")
    solver.add(
        ctx.mkEq(
            ctx.mkAdd(x, ctx.mkMul(vx, t)),
            ctx.mkAdd(ctx.mkReal(h.p.x.toLong()), ctx.mkMul(ctx.mkReal(h.v.x.toLong()), t))
        )
    )
    solver.add(
        ctx.mkEq(
            ctx.mkAdd(y, ctx.mkMul(vy, t)),
            ctx.mkAdd(ctx.mkReal(h.p.y.toLong()), ctx.mkMul(ctx.mkReal(h.v.y.toLong()), t))
        )
    )
    solver.add(
        ctx.mkEq(
            ctx.mkAdd(z, ctx.mkMul(vz, t)),
            ctx.mkAdd(ctx.mkReal(h.p.z.toLong()), ctx.mkMul(ctx.mkReal(h.v.z.toLong()), t))
        )
    )
}
if (solver.check() == Status.SATISFIABLE) {
    println(solver.model.eval(ctx.mkAdd(x, ctx.mkAdd(y, z)), false))
}